In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./web_sherlock/xg_samp_re_0.1_valid_result.csv")
df_valid = pd.read_csv("./we_data/validation.csv")

In [3]:
df.shape

(303925, 3)

In [5]:
df_valid.shape

(303925, 25)

In [6]:
df.head()

click  click_pred_per  click_pred_class
0      0        0.000199                 0
1      0        0.000127                 0
2      0        0.000127                 0
3      0        0.000140                 0
4      0        0.000111                 0

In [7]:
pCTR = df['click_pred_per'].as_matrix()

In [8]:
# parameters
# LOW = 10
# HIGH = 40
total_budget = 6250000
payprice = df_valid['payprice'].as_matrix()
Train_df = df_valid
avgCTR = 1793/2430981 #train set
#tuning...
# base_bid = 300

In [9]:
# c = 50
# lam = 0.000000052

In [10]:
# bidprices = base_bid*test_predictions/avgCTR

In [11]:
def generate_bidprice(c, lam, pCTR):
#     avgCTR = 1793/2430981#train set
    bidprices = np.sqrt(c/lam*pCTR+c*c)-c
    assert bidprices.shape == pCTR.shape
    return bidprices

In [12]:
def Prob3(Train_df,payprice,c, lam, pCTR):
    Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
         
    current_budget = total_budget
#     win_times = 0
    Train_df['win'] = 0 #initialize win label
    bidprices = generate_bidprice(c, lam, pCTR)
    #start biding...
    for i in Train_df.index:
        if current_budget > 0:
            #generate bidprice:
            current_bidprice = bidprices[i]
            
#             if current_bidprice > current_budget:
#                 print('budget remains:',current_budget)
#                 break
#                 current_bidprice = current_budget
#             bidprices.append(current_bidprice)#record

            #Winning criterion:
            current_payprice = payprice[i]
            if current_bidprice > current_payprice:
                Train_df.at[i, 'win'] = int(1)
                current_budget = current_budget - current_payprice
#                 win_times = win_times + 1
            else:
                Train_df.at[i, 'win'] = int(0)
        else:
            print('lack of budget!',current_budget)
            break
    #biding end    
    
    Train_df_new = Train_df[Train_df['win']!=0]
    #wintimes,_ = Train_df_new.shape
    #print('wintimes:',wintimes)
    #print('current_budget',current_budget)

    Clk = Train_df_new['click'].sum()
    Imp = Train_df_new['click'].count()
    CTR = Clk/Imp
    Spd = total_budget - current_budget
    CPM = 1000*Spd/Imp
    eCPC = Spd/Clk
    
    Train_Const_summ.loc[0,'Clicks']=Clk
    Train_Const_summ.loc[0,'Spend']=Spd
    Train_Const_summ.loc[0,'CTR']=CTR
    Train_Const_summ.loc[0,'CPM']=CPM
    Train_Const_summ.loc[0,'eCPC']=eCPC

    print(Train_Const_summ)
    
    
    
        
    return Train_df,current_budget

In [13]:
# Train_df,current_budget = Prob3(Train_df,payprice,c, lam, pCTR)

In [14]:
# Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
# Train_df_new = Train_df[Train_df['win']!=0]
# wintimes,_ = Train_df_new.shape
# print('wintimes:',wintimes)
# print('current_budget',current_budget)

# Clk = Train_df_new['click'].sum()
# Imp = Train_df_new['click'].count()
# CTR = Clk/Imp
# Spd = total_budget - current_budget
# CPM = 1000*Spd/Imp
# eCPC = Spd/Clk
    
# Train_Const_summ.loc[0,'Clicks']=Clk
# Train_Const_summ.loc[0,'Spend']=Spd
# Train_Const_summ.loc[0,'CTR']=CTR
# Train_Const_summ.loc[0,'CPM']=CPM
# Train_Const_summ.loc[0,'eCPC']=eCPC

# print(Train_Const_summ)

In [15]:
def calculate_click(Train_df,payprice,c, lam, pCTR):
    print('c:',c,';lambda:',lam)
    result_df,current_budget = Prob3(Train_df,payprice,c, lam, pCTR)
    print('current_budget:',current_budget)
    Train_df_new = result_df[result_df['win']!=0]
    wintimes,_ = Train_df_new.shape
    print('wintimes:',wintimes)
    print('----------------')
    Clk = Train_df_new['click'].sum()    
    return Clk

In [19]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import mpld3
import numpy as np

In [16]:
l, h = 1.371e-06, 2.296e-06
r = 0.0000001

In [21]:
cs = np.arange(20,200,10)
lams = np.arange(l,h,r)
maxClks=[]
for lam in lams:
    Clks = [calculate_click(Train_df,payprice,c, lam, pCTR) for c in cs]
    print('max@:',max(Clks))
    maxClks.append(max(Clks))
    
    fig = plt.figure()
    plt.plot(cs,Clks)
    mpld3.display(fig)
    
print('maxmax:',max(maxClks))

c: 20 ;lambda: 1.371e-06
       CPM         CTR Clicks    Spend     eCPC
0  36233.6  0.00121998    150  4455025  29700.2
current_budget: 1794975
wintimes: 122953
----------------
c: 30 ;lambda: 1.371e-06
     CPM         CTR Clicks    Spend     eCPC
0  39739  0.00116839    162  5509885  34011.6
current_budget: 740115
wintimes: 138652
----------------
c: 40 ;lambda: 1.371e-06
lack of budget! 0
       CPM         CTR Clicks    Spend     eCPC
0  42028.1  0.00111627    166  6250000  37650.6
current_budget: 0
wintimes: 148710
----------------
c: 50 ;lambda: 1.371e-06
lack of budget! -63
       CPM         CTR Clicks    Spend     eCPC
0  43802.6  0.00101621    145  6250063  43103.9
current_budget: -63
wintimes: 142687
----------------
c: 60 ;lambda: 1.371e-06
lack of budget! -109
       CPM         CTR Clicks    Spend     eCPC
0  45232.3  0.00102766    142  6250109  44014.9
current_budget: -109
wintimes: 138178
----------------
c: 70 ;lambda: 1.371e-06
lack of budget! -41
     CPM          C

lack of budget! -5
       CPM         CTR Clicks    Spend     eCPC
0  45869.9  0.00103482    141  6250005  44326.3
current_budget: -5
wintimes: 136255
----------------
c: 100 ;lambda: 1.571e-06
lack of budget! -33
       CPM        CTR Clicks    Spend     eCPC
0  46530.6  0.0010125    136  6250033  45956.1
current_budget: -33
wintimes: 134321
----------------
c: 110 ;lambda: 1.571e-06
lack of budget! -18
       CPM         CTR Clicks    Spend     eCPC
0  47108.4  0.00100246    133  6250018  46992.6
current_budget: -18
wintimes: 132673
----------------
c: 120 ;lambda: 1.571e-06
lack of budget! -55
       CPM         CTR Clicks    Spend     eCPC
0  47597.3  0.00100525    132  6250055  47348.9
current_budget: -55
wintimes: 131311
----------------
c: 130 ;lambda: 1.571e-06
lack of budget! -47
     CPM          CTR Clicks    Spend   eCPC
0  48074  0.000992239    129  6250047  48450
current_budget: -47
wintimes: 130009
----------------
c: 140 ;lambda: 1.571e-06
lack of budget! -28
       CPM

lack of budget! -83
       CPM         CTR Clicks    Spend     eCPC
0  47120.3  0.00101778    135  6250083  46296.9
current_budget: -83
wintimes: 132641
----------------
c: 170 ;lambda: 1.771e-06
lack of budget! -38
       CPM         CTR Clicks    Spend     eCPC
0  47402.6  0.00102389    135  6250038  46296.6
current_budget: -38
wintimes: 131850
----------------
c: 180 ;lambda: 1.771e-06
lack of budget! -142
       CPM         CTR Clicks    Spend     eCPC
0  47652.8  0.00102165    134  6250142  46642.9
current_budget: -142
wintimes: 131160
----------------
c: 190 ;lambda: 1.771e-06
lack of budget! -93
       CPM         CTR Clicks    Spend     eCPC
0  47900.8  0.00101931    133  6250093  46993.2
current_budget: -93
wintimes: 130480
----------------
max@: 166
c: 20 ;lambda: 1.871e-06
       CPM        CTR Clicks    Spend   eCPC
0  32229.8  0.0013894    141  3270776  23197
current_budget: 2979224
wintimes: 101483
----------------
c: 30 ;lambda: 1.871e-06
       CPM         CTR Clicks   

       CPM         CTR Clicks    Spend     eCPC
0  39255.1  0.00122353    157  5037093  32083.4
current_budget: 1212907
wintimes: 128317
----------------
c: 70 ;lambda: 2.071e-06
       CPM         CTR Clicks    Spend     eCPC
0  40219.1  0.00121872    161  5313180  33001.1
current_budget: 936820
wintimes: 132106
----------------
c: 80 ;lambda: 2.071e-06
       CPM         CTR Clicks    Spend     eCPC
0  41015.2  0.00122031    165  5545752  33610.6
current_budget: 704248
wintimes: 135212
----------------
c: 90 ;lambda: 2.071e-06
       CPM         CTR Clicks    Spend     eCPC
0  41665.2  0.00120517    166  5738966  34572.1
current_budget: 511034
wintimes: 137740
----------------
c: 100 ;lambda: 2.071e-06
     CPM         CTR Clicks    Spend   eCPC
0  42216  0.00118641    166  5906776  35583
current_budget: 343224
wintimes: 139918
----------------
c: 110 ;lambda: 2.071e-06
       CPM         CTR Clicks    Spend     eCPC
0  42729.6  0.00117063    166  6059228  36501.4
current_budget: 190

       CPM        CTR Clicks    Spend     eCPC
0  42883.8  0.0011865    165  5963635  36143.2
current_budget: 286365
wintimes: 139065
----------------
c: 160 ;lambda: 2.271e-06
       CPM         CTR Clicks    Spend     eCPC
0  43158.3  0.00117853    165  6042378  36620.5
current_budget: 207622
wintimes: 140005
----------------
c: 170 ;lambda: 2.271e-06
       CPM         CTR Clicks    Spend     eCPC
0  43403.8  0.00117842    166  6114124  36832.1
current_budget: 135876
wintimes: 140866
----------------
c: 180 ;lambda: 2.271e-06
       CPM        CTR Clicks    Spend     eCPC
0  43633.9  0.0011716    166  6182308  37242.8
current_budget: 67692
wintimes: 141686
----------------
c: 190 ;lambda: 2.271e-06
       CPM         CTR Clicks    Spend     eCPC
0  43848.9  0.00116553    166  6245134  37621.3
current_budget: 4866
wintimes: 142424
----------------
max@: 166
maxmax: 166


In [22]:
fig = plt.figure()
plt.plot(lams,maxClks)
mpld3.display(fig)

In [17]:
lam = (l+h)/2
cs = np.arange(10,500,5)
Clks = [calculate_click(Train_df,payprice,c, lam, pCTR) for c in cs]
print('max@:',max(Clks))
fig = plt.figure()
plt.plot(cs,Clks)
mpld3.display(fig)

c: 10 ;lambda: 3.0735000000000003e-06
current_budget: 5417178
wintimes: 50502
----------------
c: 15 ;lambda: 3.0735000000000003e-06
current_budget: 4936658
wintimes: 66198
----------------
c: 20 ;lambda: 3.0735000000000003e-06
current_budget: 4472616
wintimes: 77661
----------------
c: 25 ;lambda: 3.0735000000000003e-06
current_budget: 3986154
wintimes: 87488
----------------
c: 30 ;lambda: 3.0735000000000003e-06
current_budget: 3495222
wintimes: 96361
----------------
c: 35 ;lambda: 3.0735000000000003e-06
current_budget: 3061360
wintimes: 103760
----------------
c: 40 ;lambda: 3.0735000000000003e-06
current_budget: 2677565
wintimes: 109963
----------------
c: 45 ;lambda: 3.0735000000000003e-06
current_budget: 2333038
wintimes: 115279
----------------
c: 50 ;lambda: 3.0735000000000003e-06
current_budget: 2030773
wintimes: 119784
----------------
c: 55 ;lambda: 3.0735000000000003e-06
current_budget: 1769894
wintimes: 123475
----------------
c: 60 ;lambda: 3.0735000000000003e-06
current

lack of budget! -9
current_budget: -9
wintimes: 130529
----------------
c: 395 ;lambda: 3.0735000000000003e-06
lack of budget! -14
current_budget: -14
wintimes: 130479
----------------
c: 400 ;lambda: 3.0735000000000003e-06
lack of budget! -33
current_budget: -33
wintimes: 130433
----------------
c: 405 ;lambda: 3.0735000000000003e-06
lack of budget! -43
current_budget: -43
wintimes: 130378
----------------
c: 410 ;lambda: 3.0735000000000003e-06
lack of budget! -76
current_budget: -76
wintimes: 130300
----------------
c: 415 ;lambda: 3.0735000000000003e-06
lack of budget! -50
current_budget: -50
wintimes: 130258
----------------
c: 420 ;lambda: 3.0735000000000003e-06
lack of budget! -58
current_budget: -58
wintimes: 130242
----------------
c: 425 ;lambda: 3.0735000000000003e-06
lack of budget! -54
current_budget: -54
wintimes: 130182
----------------
c: 430 ;lambda: 3.0735000000000003e-06
lack of budget! -163
current_budget: -163
wintimes: 130141
----------------
c: 435 ;lambda: 3.0735

In [19]:
lam = (l+h)/2
cs = np.arange(50,150,5)
Clks = [calculate_click(Train_df,payprice,c, lam, pCTR) for c in cs]
print('max@:',max(Clks))
fig = plt.figure()
plt.plot(cs,Clks)
mpld3.display(fig)

c: 50 ;lambda: 3.0735000000000003e-06
current_budget: 2030773
wintimes: 119784
----------------
c: 55 ;lambda: 3.0735000000000003e-06
current_budget: 1769894
wintimes: 123475
----------------
c: 60 ;lambda: 3.0735000000000003e-06
current_budget: 1544659
wintimes: 126644
----------------
c: 65 ;lambda: 3.0735000000000003e-06
current_budget: 1342032
wintimes: 129362
----------------
c: 70 ;lambda: 3.0735000000000003e-06
current_budget: 1155191
wintimes: 131751
----------------
c: 75 ;lambda: 3.0735000000000003e-06
current_budget: 989599
wintimes: 133860
----------------
c: 80 ;lambda: 3.0735000000000003e-06
current_budget: 843382
wintimes: 135628
----------------
c: 85 ;lambda: 3.0735000000000003e-06
current_budget: 705789
wintimes: 137248
----------------
c: 90 ;lambda: 3.0735000000000003e-06
current_budget: 582260
wintimes: 138661
----------------
c: 95 ;lambda: 3.0735000000000003e-06
current_budget: 468232
wintimes: 139967
----------------
c: 100 ;lambda: 3.0735000000000003e-06
curren

In [20]:
lam = (l+h)/2
cs = np.arange(100,121,1)
Clks = [calculate_click(Train_df,payprice,c, lam, pCTR) for c in cs]
print('max@:',max(Clks))
fig = plt.figure()
plt.plot(cs,Clks)
mpld3.display(fig)

c: 100 ;lambda: 3.0735000000000003e-06
current_budget: 358138
wintimes: 141161
----------------
c: 101 ;lambda: 3.0735000000000003e-06
current_budget: 337707
wintimes: 141384
----------------
c: 102 ;lambda: 3.0735000000000003e-06
current_budget: 317255
wintimes: 141602
----------------
c: 103 ;lambda: 3.0735000000000003e-06
current_budget: 299676
wintimes: 141795
----------------
c: 104 ;lambda: 3.0735000000000003e-06
current_budget: 278371
wintimes: 142023
----------------
c: 105 ;lambda: 3.0735000000000003e-06
current_budget: 259533
wintimes: 142227
----------------
c: 106 ;lambda: 3.0735000000000003e-06
current_budget: 238875
wintimes: 142436
----------------
c: 107 ;lambda: 3.0735000000000003e-06
current_budget: 220214
wintimes: 142648
----------------
c: 108 ;lambda: 3.0735000000000003e-06
current_budget: 202464
wintimes: 142837
----------------
c: 109 ;lambda: 3.0735000000000003e-06
current_budget: 184839
wintimes: 143020
----------------
c: 110 ;lambda: 3.0735000000000003e-06
c

In [22]:
#result
lam = (l+h)/2
c = 110
print(lam)
print(c)

3.0735000000000003e-06
110


In [17]:
cs = np.arange(10,500,5)
lams = np.arange(l,h,0.00000005)
maxClks=[]
for lam in lams:
    Clks = [calculate_click(Train_df,payprice,c, lam, pCTR) for c in cs]
    print('max@:',max(Clks))
    maxClks.append(max(Clks))
    
    fig = plt.figure()
    plt.plot(cs,Clks)
    mpld3.display(fig)
    
print('maxmax:',max(maxClks))

c: 10 ;lambda: [2.881e-06 2.931e-06 2.981e-06 3.031e-06 3.081e-06 3.131e-06 3.181e-06
 3.231e-06]


ValueError: operands could not be broadcast together with shapes (8,) (303925,) 

In [ ]:
#thisisend

In [66]:
# parameters
# LOW = 10
# HIGH = 40
total_budget = 6250000
payprice = df_valid['payprice'].as_matrix()
Train_df = df_valid
avgCTR = 1793/2430981 #train set
#tuning...
base_bid = 68

In [67]:
Train_df,current_budget = Prob3(Train_df,payprice,base_bid,test_predictions)

In [68]:
Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
Train_df_new = Train_df[Train_df['win']!=0]
wintimes,_ = Train_df_new.shape
print('wintimes:',wintimes)
print('current_budget',current_budget)

Clk = Train_df_new['click'].sum()
Imp = Train_df_new['click'].count()
CTR = Clk/Imp
Spd = total_budget - current_budget
CPM = 1000*Spd/Imp
eCPC = Spd/Clk
    
Train_Const_summ.loc[0,'Clicks']=Clk
Train_Const_summ.loc[0,'Spend']=Spd
Train_Const_summ.loc[0,'CTR']=CTR
Train_Const_summ.loc[0,'CPM']=CPM
Train_Const_summ.loc[0,'eCPC']=eCPC

print(Train_Const_summ)

wintimes: 120965
current_budget 253018
       CPM          CTR Clicks    Spend     eCPC
0  49576.2  0.000810152     98  5996982  61193.7


In [9]:
# #字典中的key值即为csv中列名
# dataframe = pd.DataFrame({'bidprice':bidprices})

# #将DataFrame存储为csv,index表示是否显示行名，default=True
# dataframe.to_csv("bidprice10-40.csv",index=False,sep=',')

In [ ]:
# group_df = pd.read_csv('/Users/qrdai/web_project/we_data/train.csv')